In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist["data"], mnist["target"]
X = np.array(X)
y = np.array(y)

In [18]:
y = y.astype(np.uint8)

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [6]:
enc.fit(y[:,np.newaxis])

OneHotEncoder()

In [7]:
Y = enc.transform(y[:,np.newaxis]).toarray()

In [8]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], Y[:60000], Y[60000:]

In [9]:
X_train = X_train / 255
X_test = X_test / 255

In [22]:
np.save('./x_train', X_train)
np.save('./x_test', X_test)
np.save('./y_train', y_train)
np.save('./y_test', y_test)

In [1]:
import numpy as np

X_train = np.load('./x_train.npy')
X_test = np.load('./x_test.npy')
y_train = np.load('./y_train.npy')
y_test = np.load('./y_test.npy')

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def softmax(X, W):
    K = np.size(W, 1)
    A = np.exp(X @ W)
    B = np.diag(1 / (np.reshape(A @ np.ones((K,1)), -1)))
    Y = B @ A
    return Y

In [4]:
def compute_cost(X, T, W):
    epsilon = 1e-5
    N = len(T)
    K = np.size(T, 1)
    cost = - (1/N) * np.ones((1,N)) @ (np.multiply(np.log(softmax(X, W) + epsilon), T)) @ np.ones((K,1))
    return cost

In [5]:
def predict(X, W):
    return np.argmax((X @ W), axis=1)

In [6]:
def batch_gd(X, T, W, learning_rate, iterations, batch_size):
    N = len(T)
    cost_history = np.zeros((iterations,1))
    shuffled_indices = np.random.permutation(N)
    X_shuffled = X[shuffled_indices]
    T_shuffled = T[shuffled_indices]

    for i in range(iterations):
        j = i % N
        X_batch = X_shuffled[j:j+batch_size]
        T_batch = T_shuffled[j:j+batch_size]
        # batch가 epoch 경계를 넘어가는 경우, 앞 부분으로 채워줌
        if X_batch.shape[0] < batch_size:
            X_batch = np.vstack((X_batch, X_shuffled[:(batch_size - X_batch.shape[0])]))
            T_batch = np.vstack((T_batch, T_shuffled[:(batch_size - T_batch.shape[0])]))
        W = W - (learning_rate/batch_size) * (X_batch.T @ (softmax(X_batch, W) - T_batch))
        cost_history[i] = compute_cost(X_batch, T_batch, W)
        if i % 1000 == 0:
            print(cost_history[i][0])

    return (cost_history, W)

In [7]:
X = np.hstack((np.ones((np.size(X_train, 0),1)),X_train))
T = y_train

K = np.size(T, 1)
M = np.size(X, 1)
W = np.zeros((M,K))

iterations = 50000
learning_rate = 0.01

initial_cost = compute_cost(X, T, W)

In [8]:
print("Initial Cost is: {} \n".format(initial_cost[0][0]))

Initial Cost is: 2.3024850979937166 



In [9]:
(cost_history, W_optimal) = batch_gd(X, T, W, learning_rate, iterations, 64)

2.279072912726263
0.47705775666480116
0.4960997979964924
0.4901447292838354
0.26786155327310773
0.21093095823439742
0.23092033601563397
0.27071681212536036
0.3869465901657288
0.2133830970202236
0.43275954846919795
0.3528659239227844
0.3477505276766501
0.16615652357005695
0.29839248518747186
0.32030109619105246
0.33209436344894677
0.4112691138165702
0.26281593221743643
0.21387971001827738
0.26283505166719484
0.4757718178103155
0.24838655931688705
0.3059738943422072
0.23048730744914533
0.3620429831620927
0.24601117412887918
0.23372937730978716
0.2467647090477123
0.20425613527206107
0.17643993687812942
0.18941274708436545
0.20675267080217732
0.2572080111514307
0.3364381360007028
0.2609069656809952
0.4614984004956031
0.1322453059558878
0.45273062677058695
0.3081555612395968
0.2578183271298885
0.2081675837015743
0.25871244111429675
0.2805862315418276
0.1565011907031097
0.28280121547767373
0.20346745337748937
0.17206552168675232
0.20922352530135824
0.13107514057970016


In [10]:
## Accuracy
X_ = np.hstack((np.ones((np.size(X_test, 0),1)),X_test))
T_ = y_test
y_pred = predict(X_, W_optimal)
score = float(sum(y_pred == np.argmax(T_, axis=1)))/ float(len(y_test))

print(score)

0.9142
